# QMMM workflow using LAMMPS and VOTCA-XTP

## What is this tutorial about
In this tutorial, we will learn how to set and perform excited state calculation using the Votca XTP library. We will use thiophene as our QM region.

## Requirements
* You will need to install **VOTCA** using the instructions described [here](https://github.com/votca/votca/blob/master/share/doc/INSTALL.md)
* Once the installation is completed you need to activate the VOTCA enviroment by running the `VOTCARC.bash` script that has been installed at the bin subfolder for the path that you have provided for the installation step above

## Interacting with the XTP command line interface
The XTP package offers the following command line interface that the user can interact with:
* [xtp_map](https://votca.github.io/xtp_map.html)
* [xtp_parallel](https://votca.github.io/xtp_parallel.html)
* [xtp_run](https://votca.github.io/xtp_run.html)
* [xtp_tools](https://votca.github.io/xtp_tools.html)

Run the following command to view the help message of `xtp_tools`:

In [1]:
!xtp_tools -h

========   VOTCA (http://www.votca.org)   ========

please submit bugs to bugs@votca.org

xtp_tools, version 1.7-dev gitid: 6e47b87 (dirty) (compiled Jun  2 2020, 10:18:25)
votca_csg, version 1.7-dev gitid: 6e47b87 (dirty) (compiled Jun  2 2020, 10:14:08)
votca_tools, version 1.7-dev gitid: 6e47b87 (dirty) (compiled Jun  2 2020, 10:13:13)

Runs excitation/charge transport tools



Allowed options:
  -h [ --help ]                 display this help and exit
  --verbose                     be loud and noisy
  --verbose1                    be very loud and noisy
  -v [ --verbose2 ]             be extremly loud and noisy
  -o [ --options ] arg          calculator options
  -t [ --nthreads ] arg (=1)    number of threads to create

Tools:
  -e [ --execute ] arg        List of tools separated by ',' or ' '
  -l [ --list ]               Lists all available tools
  -d [ --description ] arg    Short description of a tool
  -n [ --name ] arg           Name of the job to run



### Note
> * In Jupyter the `!` symbol means: *run the following command as a standard unix command*
> * In Jupyter the command `%env` set an environmental variable

## Setting the environment


Create a folder to store the input `Options` for XTP

In [2]:
!mkdir -p OPTIONFILES

Remove previous hdf5 file

In [3]:
!rm -f state.hdf5

Import an small python module to edit xml files

In [4]:
from xml_editor import add_section, edit_calculator
import os

## Generate the topology from the Gromacs file
We will first generate the mapping from MD coordinates to segments, creating an [hdf5 file](https://www.hdfgroup.org/solutions/hdf5/) to store the results. You can explore the generated `state.hdf5` file with e.g. hdf5itebrowser. In Python, you can use the [h5py library](https://www.h5py.org/). The command to generate the mapping is the following,

In [5]:
!xtp_map -v -t MD_FILES/newfile.data -c MD_FILES/traj.dump -s system.xml -f state.hdf5 -i 99 > mapping.out

##  Check the mapping

Let us first output `.pdb` files for the segments, qmmolecules and classical segments in order to check the mapping. We will need to copy the input from the `VOTCA` installation path to our local copy.

In [6]:
!cp "$VOTCASHARE/xtp/xml/mapchecker.xml" OPTIONFILES/

The `${VOTCASHARE}` environmental variable is set to the path that you provided during the `VOTCA` [installation](https://github.com/votca/votca/blob/master/share/doc/INSTALL.md), by the default is set to `/usr/local/votca`.

We need to update the `mapchecker` options with the file containing the definition of the system. For doing, so we will use the python module `xml_editor` that we have imported above. The first argument is the name of the calculator (i.e. `mapchecker`), the seconds argument is the name of the option to change and the third one, the value to use.

In [7]:
edit_calculator("mapchecker", "map_file", "system.xml")

The option 'map_file' on file 'OPTIONFILES/mapchecker.xml' has been set to 'system.xml'


In the [mapchecker section of the manual](https://votca.github.io/mapchecker.html) you can find a table with the `mapchecker` input variables and their corresponding defaults. Finally, the following command run the check

In [8]:
!xtp_run -e mapchecker -o OPTIONFILES/mapchecker.xml -f state.hdf5 > mapchecker.log

## Neighborlist Calculation
The following step is to determine the neighbouring pairs for exciton transport. We will use the [default options for the neighborlist](https://votca.github.io/neighborlist.html) calculator,

In [9]:
!cp "$VOTCASHARE/xtp/xml/neighborlist.xml" OPTIONFILES/
!xtp_run -e neighborlist -o OPTIONFILES/neighborlist.xml -f state.hdf5 > neighborlist.log

## Read reorganization energies
In this step we will read the in site reorganization energies and store them in the `state.hdf5` file. We just need to copy the input file and execute the calculation

In [10]:
!cp "$VOTCASHARE/xtp/xml/einternal.xml" OPTIONFILES/
!xtp_run -e einternal -o OPTIONFILES/einternal.xml -f state.hdf5  > einternal.log

## Compute site energy
In this step we will perform some *QMMM* calculations to compute the site energies. The `qmmm_mm.xml` file contains some predefined settings to perform the *MM* calculations. Let us first copy these settings into the state file,

In [11]:
!cp qmmm_mm.xml OPTIONFILES
!xtp_parallel -e qmmm -o OPTIONFILES/qmmm_mm.xml -f state.hdf5 -j "write" > qmmm_mm.log

The previous command generates a `qmmm_mm_jobs.xml` containing 3000 *MM* jobs to compute, if you examine that file, it should look something like:
```xml
<jobs>
        <job>
                <id>0</id>
                <tag>thiophene_0:n</tag>
                <input>
                        <site_energies>0:n</site_energies>
                        <regions>
                                <region>
                                        <id>0</id>
                                        <segments>0:n</segments>
                                </region>
                        </regions>
                </input>
                <status>AVAILABLE</status>
        </job>
```

Let us run just the first 4 jobs by settings all jobs `status` to `COMPLETE` except for the first four. This can be easily done with [sed](https://www.gnu.org/software/sed/manual/sed.html) as follows,

In [12]:
!sed -i "s/AVAILABLE/COMPLETE/g" qmmm_mm_jobs.xml
!sed -i '0,/COMPLETE/s/COMPLETE/AVAILABLE/' qmmm_mm_jobs.xml
!sed -i '0,/COMPLETE/s/COMPLETE/AVAILABLE/' qmmm_mm_jobs.xml
!sed -i '0,/COMPLETE/s/COMPLETE/AVAILABLE/' qmmm_mm_jobs.xml
!sed -i '0,/COMPLETE/s/COMPLETE/AVAILABLE/' qmmm_mm_jobs.xml

Now we can run the jobs and save the results in the state file

In [13]:
!xtp_parallel -e qmmm -o OPTIONFILES/qmmm_mm.xml -f state.hdf5 -j "run"  >> qmmm_mm.log
!xtp_parallel -e qmmm -o OPTIONFILES/qmmm_mm.xml -f state.hdf5 -j "read" >> qmmm_mm.log

## Site energy and pair energy analysis
In this step we generate an histogram and compute the correlation function of site energies and pair energy differences.

In [14]:
!cp "$VOTCASHARE/xtp/xml/eanalyze.xml" OPTIONFILES/
!xtp_run -e eanalyze -o OPTIONFILES/eanalyze.xml -f state.hdf5 > eanalyze.log

You should now see a set of files prefixed with `eanalyze` containing the histrogram and correlation functions.

In [15]:
!ls eanalyze*

eanalyze.log		 eanalyze.pairlist_h.out  eanalyze.sitecorr_t.out
eanalyze.pairhist_e.out  eanalyze.pairlist_s.out  eanalyze.sitehist_e.out
eanalyze.pairhist_h.out  eanalyze.pairlist_t.out  eanalyze.sitehist_h.out
eanalyze.pairhist_s.out  eanalyze.sitecorr_e.out  eanalyze.sitehist_s.out
eanalyze.pairhist_t.out  eanalyze.sitecorr_h.out  eanalyze.sitehist_t.out
eanalyze.pairlist_e.out  eanalyze.sitecorr_s.out


## QM energy calculation
Our next task is to perform the qm calculations for each segment that we have stored in the hdf5 file. The calculations take place in 3 stages: write the jobs to a file, perform the computation and finally save the results to the state file. We will first copy the input into our local folder

In [16]:
!cp "$VOTCASHARE/xtp/xml/eqm.xml" OPTIONFILES/

Now we set the GWBSE mode to `G0W0`,  the `ranges` to `full` and the `basisset` and `auxbasisset` to `def2-svp` and `aux-def2-svp`. For more information, check the [eqm calculator options](https://votca.github.io/eqm.html).

In [17]:
edit_calculator("eqm", "gwbse_options.gwbse.ranges", "full")
edit_calculator("eqm", "gwbse_options.gwbse.mode", "G0W0")
edit_calculator("eqm", "map_file", "system.xml")
edit_calculator("eqm", "basisset", "def2-svp")
edit_calculator("eqm", "auxbasisset", "aux-def2-svp")

The option 'gwbse_options.gwbse.ranges' on file 'OPTIONFILES/eqm.xml' has been set to 'full'
The option 'gwbse_options.gwbse.mode' on file 'OPTIONFILES/eqm.xml' has been set to 'G0W0'
The option 'map_file' on file 'OPTIONFILES/eqm.xml' has been set to 'system.xml'
The option 'basisset' on file 'OPTIONFILES/eqm.xml' has been set to 'def2-svp'
The option 'auxbasisset' on file 'OPTIONFILES/eqm.xml' has been set to 'aux-def2-svp'


For the sake of computational time let just compute the `gw` approximation and the `singlet`. You can also request the `triplet` or `all`,  see the [gwbse sectionfor the eqm calculator](https://votca.github.io/eqm.html).

In [18]:
edit_calculator("eqm", "gwbse_options.gwbse.tasks", "gw,singlets")

The option 'gwbse_options.gwbse.tasks' on file 'OPTIONFILES/eqm.xml' has been set to 'gw,singlets'


First we will write the job in a file and enable only the first 2 jobs

In [19]:
!xtp_parallel -e eqm -o OPTIONFILES/eqm.xml -f state.hdf5 -s 0 -j "write" > eqm.log
!sed -i "s/AVAILABLE/COMPLETE/g" eqm.jobs
!sed -i '0,/COMPLETE/s/COMPLETE/AVAILABLE/' eqm.jobs
!sed -i '0,/COMPLETE/s/COMPLETE/AVAILABLE/' eqm.jobs

Now, let run these 2 jobs

In [20]:
!xtp_parallel -e eqm -o OPTIONFILES/eqm.xml -f state.hdf5 -s 0 -j run -c 1 -p 4 >> eqm.log

## QM calculation for pairs
In the following step we will run QM calculations for each pair in the hdf5 file. As the calculations on the previous step, we will first write the jobs in a file, then run them and finally store the results in the state file. First, we need to copy the input to our local folder

In [21]:
!cp "$VOTCASHARE/xtp/xml/iqm.xml" OPTIONFILES/

As in the previous section, we set the GWBSE mode to `G0W0`and the `ranges` to `full`, but we compute only the `gw` approximation. We are also going to compute a single excited state. For more information, check the [iqm calculator options](https://votca.github.io/iqm.html). We also want to compute the `singlet` couplings. 

In [22]:
edit_calculator("iqm", "states", "1")
edit_calculator("iqm", "map_file", "system.xml")
edit_calculator("iqm", "gwbse_options.gwbse.ranges", "full")
edit_calculator("iqm", "gwbse_options.gwbse.tasks", "gw")
edit_calculator("iqm", "bsecoupling.spin", "singlet")
edit_calculator("iqm", "basisset", "def2-svp")
edit_calculator("iqm", "auxbasisset", "aux-def2-svp")

The option 'states' on file 'OPTIONFILES/iqm.xml' has been set to '1'
The option 'map_file' on file 'OPTIONFILES/iqm.xml' has been set to 'system.xml'
The option 'gwbse_options.gwbse.ranges' on file 'OPTIONFILES/iqm.xml' has been set to 'full'
The option 'gwbse_options.gwbse.tasks' on file 'OPTIONFILES/iqm.xml' has been set to 'gw'
The option 'bsecoupling.spin' on file 'OPTIONFILES/iqm.xml' has been set to 'singlet'
The option 'basisset' on file 'OPTIONFILES/iqm.xml' has been set to 'def2-svp'
The option 'auxbasisset' on file 'OPTIONFILES/iqm.xml' has been set to 'aux-def2-svp'


Before running the calculations, we need to specify in the `iqm` input which states to read into the jobfile for each segment type. The following *XML* snippet needs to be added

In [23]:
readjobfile = """
<readjobfile help="which states to read into the jobfile for each segment type">
     <singlet>thiophene:s1</singlet>
     <triplet>thiophene:t1</triplet>
     <electron>thiophene:e1</electron>
     <hole>thiophene:h1</hole>
</readjobfile>
"""

The following function add the `readjobfile` section,

In [24]:
add_section("iqm", readjobfile)

Now, let's write the jobs to the file

In [25]:
!xtp_parallel -e iqm -o OPTIONFILES/iqm.xml -f state.hdf5 -s 0 -j "write" > iqm.log

From the jobs that we just write down, let's make available only the first job

In [26]:
!sed -i "s/AVAILABLE/COMPLETE/g" iqm.jobs
!sed -i '0,/COMPLETE/s/COMPLETE/AVAILABLE/' iqm.jobs

Now we can run and store the jobs results

In [27]:
!xtp_parallel -e iqm -o OPTIONFILES/iqm.xml -f state.hdf5 -s 0 -j run -c 1 -p 4 >> iqm.log

Finally, we read the results into the state

In [28]:
!xtp_parallel -e iqm -o OPTIONFILES/iqm.xml -f state.hdf5 -j "read">> iqm.log

## QMMM Calculations
We will run the *QMMM* calculations we will use the pregenerated `qmmm.jobs` file in the current work directory, so we cana directly run the calculations

In [29]:
!cp qmmm.xml OPTIONFILES
!xtp_parallel -e qmmm -o OPTIONFILES/qmmm.xml -f state.hdf5 -j run -p 4 > qmmm.log